In [1]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.agents.tools.db_tools import DBToolManager
# Test tool manager
tool_mgr = DBToolManager()
tools = tool_mgr.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])

tool_mgr = DBToolManager()
list_tool = tool_mgr.get_tool_by_name("list_all_tables")
print(list_tool.invoke({}))

🔄 Initializing DB connections...


c:\Users\sourabh.gupta\Documents\Workspace\personal-git-workspace\NL_DB_Q\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'DatabaseUser'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\personal-git-workspace\NL_DB_Q\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Event'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\personal-git-workspace\NL_DB_Q\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Schema'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\personal-git-workspace\NL_DB_Q\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Object'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\

✅ DB connections initialized (pooled)
✅ 4 tools ready
Tool names: ['list_all_tables', 'get_table_schema', 'preview_sql', 'execute_sql']
dbo.AWBuildVersion, dbo.DatabaseLog, dbo.ErrorLog, Sales.CountryRegionCurrency, Sales.CreditCard, Sales.Currency, Sales.CurrencyRate, Sales.Customer, Sales.PersonCreditCard, Sales.SalesOrderDetail, Sales.SalesOrderHeader, Sales.SalesOrderHeaderSalesReason, Sales.SalesPerson, Sales.SalesPersonQuotaHistory, Sales.SalesReason, Sales.SalesTaxRate, Sales.SalesTerritory, Sales.SalesTerritoryHistory, Sales.ShoppingCartItem, Sales.SpecialOffer, Sales.SpecialOfferProduct, Sales.Store, Person.Address, Person.AddressType, Person.BusinessEntity, Person.BusinessEntityAddress, Person.BusinessEntityContact, Person.ContactType, Person.CountryRegion, Person.EmailAddress, Person.Password, Person.Person, Person.PersonPhone, Person.PhoneNumberType, Person.StateProvince, Production.BillOfMaterials, Production.Culture, Production.Document, Production.Illustration, Productio

In [8]:
query_tool = tool_mgr.get_tool_by_name("execute_sql")
print(query_tool.invoke({"query": "SELECT TOP(10) * FROM HumanResources.Employee"}))

❌ **Execution failed:** (pyodbc.ProgrammingError) ('ODBC SQL type -151 is not yet supported.  column-index=3  type=-151', 'HY106')
(Background on this error at: https://sqlalche.me/e/20/f405)
